# ImpDAR ApRES Differencing

Here, we define a separate data object for differencing two ApRES acquisitions. This should be done after the initial processing for each aquisition, so have a look at the opening tutorial notebook first.

The main processing functionality within the differencing class includes: 
- phase coherence
- phase unwrapping
- strain rate calculation
- bed reflector isolation

We overview each of these below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from impdar.lib.ApresData._ApresDataDifferencing import ApresDiff
%matplotlib inline

# Load the two data files together into one ApresDiff object
dat = ApresDiff('./data/diffdata_time1.mat','./data/diffdata_time2.mat')

plt.figure()
plt.plot(dat.Rcoarse,10.*np.log10(dat.data1**2.),'k',lw=2,label='year1')
plt.plot(dat.Rcoarse,10.*np.log10(dat.data2**2.),'indianred',lw=.5,label='year2')
plt.legend()
plt.xlim(0,2500)
plt.ylabel('Power (dB)')
plt.xlabel('Range (m)')

## Phase Coherence

The coherence between two acquisitions is calculated using a correlation coefficient on samples within a moving window using the function, range_diff(). The output from this function gives: the depths at the center of the window for each step, the coherence resulting from the correlation at each step, the range difference between acquisitions, and the uncertainty.

This coherence value can be used to calculate the range difference (one of the outputs as stated above), but can also be used for polarimetric coherence (e.g. Jordan et al., 2020).

The default uncertainty calculation in this case is to use the Cramer Rao bound (e.g. Jordan et al., 2020). Another option though, is to use the 'noise-phasor' uncertainty as above calculated in each acquisition and added together for the total uncertainty. This is an option in the range_diff() function.

In [ ]:
# Initial calculation of the phase difference between profiles
win = 20  # window over which the phase signal are compared
step = 20 # step size for the window moving down the profile
dat.phase_diff(win,step)

plt.figure()
plt.subplot(211)
plt.tick_params(labelbottom=False)
plt.plot(dat.ds,abs(dat.co),'k.')
plt.xlim(0,2500)
plt.ylabel('Coherence')

plt.subplot(212)
plt.plot(dat.ds,np.angle(dat.co),'k.')
plt.xlim(0,2500)
plt.ylabel('Phase Offset')
plt.xlabel('Range (m)')

## Phase Unwrap

Automatic unwrapping is done trace-by-trace using a window and a coherence threshold. If the phase of a given trace is more than $\pi$ from the previous trace and all traces within the neighboring window are above the threshold then the phase should be moved by a full cycle, $2\pi$.

## Range Conversion

Convert the phase difference to a vertical velocity profile and calculate uncertainty in vertical velocity.

## Strain Rate

Use a linear regressor from <scipy.stats> to find the slope of the velocity profile (average strain rate) within some defined window.

In [ ]:
# Unwrap the phase profile
dat.phase_unwrap(win=20,thresh=.95)
# Convert to range
dat.range_diff()
# Calculate strain rate from the profile and reference the velocity profile at the surface
w_surf = -0.15
dat.strain_rate(strain_window=(200,1000),w_surf=w_surf)

plt.figure(figsize=(6,8),facecolor='w')
plt.subplot(411)
plt.tick_params(labelbottom=False)
plt.plot(dat.ds,dat.phi,'k.')
plt.xlim(0,2500)
plt.yticks(np.arange(0.,4.01*np.pi,2.*np.pi))
plt.gca().set_yticklabels(['0','2π','4π'])
plt.ylabel('Unwrapped Phase')

plt.subplot(412)
plt.plot(dat.ds,dat.w,'k.')
plt.plot(dat.ds,dat.ds*dat.eps_zz+w_surf,'r')
plt.text(100,-.05,'Strain Rate: '+str(round(dat.eps_zz*1e5))+'x10^5',color='r',rotation=25)
plt.xlim(0,2500)
plt.ylim(-.2,.2)
plt.ylabel('Vertical velocity (m/yr)')
plt.xlabel('Range (m)')

# Plot the uncertainty calculated from the Cramer-Rao Bound (Jordan et al., 2020)
plt.subplot(413)
plt.tick_params(labelbottom=False)
plt.xlim(0,3500)
plt.ylim(0,.1)
plt.plot(dat.ds,dat.w_err,'k.',ms=2)
plt.ylabel('C-R (m)');

# Calculate and plot the uncertainty from the noise phasor as in Kingslake et al. (2014)
dat.range_diff(uncertainty='noise_phasor')

plt.subplot(414)
plt.xlim(0,3500)
plt.ylim(0,.1)
plt.plot(dat.ds,dat.w_err,'k.',ms=1,alpha=0.5)
plt.ylabel('Kings. Unc. (m)');
plt.xlabel('Depth (m)');

## Isolating the bed reflector

Automatically extract the range and power of the bed reflector. Use the <scipy.signal.find_peaks> algorithm to get the lowest 'peak' from both acquisitions and make sure that it has relatively high coherence.

In [ ]:
# Isolate the bed reflector using the standard input
dat.bed_pick()

plt.figure()
plt.plot(dat.Rcoarse,10.*np.log10(dat.data1**2.),'k',lw=2,label='year1')
plt.plot(dat.Rcoarse,10.*np.log10(dat.data2**2.),'indianred',lw=.5,label='year2')
plt.plot(dat.bed[1],dat.bed[3],'k.',mfc='w',ms=10,mew=2,label='bed pick')
plt.legend()
plt.xlim(0,2500)
plt.ylabel('Power (dB)')
plt.xlabel('Range (m)')